In [1]:
import pandas as pd
import numpy as np
import warnings
import re
warnings.filterwarnings(action='ignore')

### Lectura del csv, ajuste de variables y división del DF por temporadas

In [2]:
# Importamos el csv con la información de los partidos a nivel de jugador

jugadores = pd.read_csv('Stats_jugadores_10_04_20.csv')


# Modificamos el formato de algunas variables

jugadores['Year'] = jugadores['Year'].astype(int)
jugadores['Season'] = jugadores['Season'].astype(int)

jugadores['Team Points'] = jugadores['Team Points'].astype(int)
jugadores['Opponent Points'] = jugadores['Opponent Points'].astype(int)
jugadores['Result'] = np.where(jugadores['Team Points'] > jugadores['Opponent Points'], 'W', 'L')

jugadores = jugadores.drop(columns=['Date'])
jugadores['Date']=jugadores['ID Partido'].str.extract(pat='(2\w{7})')


# Dividimos el DF a nivel de temporada para obtener el mejor jugador por equipo en cada una de ellas

jugadoresAWS = jugadores.loc[jugadores['Season'] > 2016]


print(jugadoresAWS.head(5)) 

                      Team  Season  Year                    ID Partido  Local  \
179818  Philadelphia 76ers    2017  2016  /boxscores/201610260PHI.html   True   
179819  Philadelphia 76ers    2017  2016  /boxscores/201610260PHI.html   True   
179820  Philadelphia 76ers    2017  2016  /boxscores/201610260PHI.html   True   
179821  Philadelphia 76ers    2017  2016  /boxscores/201610260PHI.html   True   
179822  Philadelphia 76ers    2017  2016  /boxscores/201610260PHI.html   True   

                     Opponent  Team Points  Opponent Points Result  \
179818  Oklahoma City Thunder           97              103      L   
179819  Oklahoma City Thunder           97              103      L   
179820  Oklahoma City Thunder           97              103      L   
179821  Oklahoma City Thunder           97              103      L   
179822  Oklahoma City Thunder           97              103      L   

                    Name  ... Total rebound percentage  Assist percentage  \
179818  Covingt

### Obtenemos la medida de valoración compuesta "Alternative WinScore"

In [4]:
# Renombramos las variables que vamos a usar para nuestro calculo del AWS

jugadoresAWS.rename(columns={'Tiros anotados': 'CCC', 'Tiros intentados': 'CCI', 'Tiros libres anotados': 'C1C',
                                  'Tiros libres intentados': 'C1I', 'Rebotes ofensivos': 'RO', 'Rebotes defensivo': 'RD',
                                  'Asistencias': 'As', 'Robos': 'BR', 'Tapones': 'TF', 'Perdidas': 'BP', 'Faltas': 'FPC', 
                                  'Puntos': 'Pts'}, inplace=True)

jugadoresAWS = jugadoresAWS.loc[:, ['Team', 'Season', 'Date', 'ID Partido', 'Local', 'Opponent', 'Result', 'Name', 
                               'Minutes Played', 'CCC', 'CCI', 'C1C', 'C1I', 'RO', 'RD', 'As', 'BR', 'TF', 'BP', 'FPC', 'Pts']]


# Calculamos el AWS

jugadoresAWS['AWS'] = jugadoresAWS['Pts'] + jugadoresAWS['BR'] + jugadoresAWS['BP'] + \
                           + 0.5 * (jugadoresAWS['As'] + jugadoresAWS['TF'] - jugadoresAWS['FPC']) + \
                           + 0.7 * (jugadoresAWS['RO'] - jugadoresAWS['CCI']) + \
                           + 0.3 * (jugadoresAWS['RD'] - jugadoresAWS['CCC']) - 0.35 * jugadoresAWS['C1I'] - \
                           + 0.15 * jugadoresAWS['C1C']


# Obtenemos el AWS por jugador para la temporada

jugadores_AWS_MEAN = jugadoresAWS.groupby(['Team','Name', 'Season'], as_index=False)['AWS'].agg({'AWS_MEAN':'mean'})


# Obtenemos la estrella del equipo. Es decir, aquel jugador que tiene mayor AWS medio en cada equipo

AWS_Star = jugadores_AWS_MEAN.groupby(['Team', 'Season'])['AWS_MEAN'].transform(max) == jugadores_AWS_MEAN['AWS_MEAN']

print('Dataframe de jugador estrella de cada equipo por temporada: ')
jugadores_AWS_MEAN[AWS_Star]


Dataframe de jugador estrella de cada equipo por temporada: 


,Team,Name,Season,AWS_MEAN
37,Atlanta Hawks,"Howard,Dwight",2017,11.918125
65,Atlanta Hawks,"Schröder,Dennis",2018,10.377612
80,Atlanta Hawks,"Young,Trae",2019,13.282099
81,Atlanta Hawks,"Young,Trae",2020,19.123333
109,Boston Celtics,"Irving,Kyrie",2018,13.425833
110,Boston Celtics,"Irving,Kyrie",2019,14.457237
138,Boston Celtics,"Tatum,Jayson",2020,12.644068
142,Boston Celtics,"Thomas,Isaiah",2017,14.796154
194,Brooklyn Nets,"Irving,Kyrie",2020,15.425000
206,Brooklyn Nets,"Lopez,Brook",2017,11.010000


#### En el siguiente dataframe tenemos el valor del AWS a nivel de jugador en cada partido.
#### Tener en cuenta que el AWS puede ser negativo. Tenemos el dato del Id del partido para identificar de qué partido se trata. 


In [5]:
print('Dataframe del cálculo del AWS para cada jugador en cada partido: ')

jugadoresAWS.head(5)

Dataframe del cálculo del AWS para cada jugador en cada partido: 


,Team,Season,Date,ID Partido,Local,Opponent,Result,Name,Minutes Played,CCC,...,C1I,RO,RD,As,BR,TF,BP,FPC,Pts,AWS
179818,Philadelphia 76ers,2017,20161026,/boxscores/201610260PHI.html,True,Oklahoma City Thunder,L,"Covington,Robert",31:56,3.0,...,2.0,1.0,4.0,2.0,2.0,1.0,0.0,1.0,10.0,8.80
179819,Philadelphia 76ers,2017,20161026,/boxscores/201610260PHI.html,True,Oklahoma City Thunder,L,"Rodríguez,Sergio",31:51,5.0,...,0.0,0.0,4.0,9.0,0.0,0.0,0.0,1.0,12.0,8.00
179820,Philadelphia 76ers,2017,20161026,/boxscores/201610260PHI.html,True,Oklahoma City Thunder,L,"Šarić,Dario",27:27,2.0,...,2.0,1.0,6.0,2.0,0.0,0.0,2.0,3.0,5.0,-0.85
179821,Philadelphia 76ers,2017,20161026,/boxscores/201610260PHI.html,True,Oklahoma City Thunder,L,"Henderson,Gerald",24:47,3.0,...,2.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,9.0,2.65
179822,Philadelphia 76ers,2017,20161026,/boxscores/201610260PHI.html,True,Oklahoma City Thunder,L,"Embiid,Joel",22:25,6.0,...,8.0,0.0,7.0,0.0,0.0,2.0,4.0,4.0,20.0,8.25
